# Assistant testing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pprint

from npc.apis.gmail_client import GmailClient, GmailSearchOptions
from npc.apis.llm_client import Model
from npc.simulators.ai_assistant.tools.email_briefing_generator import EmailBriefingGenerator
from npc.simulators.ai_assistant.tools.email_summarizer import EmailSummarizer
from npc.simulators.ai_assistant.tools.inbox_manager import InboxManager

In [3]:
gmail_client = GmailClient()

max_emails = 50
unread_inbox = gmail_client.retrieve_emails(
    GmailSearchOptions(unread=True),
    max_results=max_emails,
)

def email_str(email):
    return f"{email.subject}\nFrom: {email.sender}"
print("\n\n".join([email_str(email) for email in unread_inbox]))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=285282244977-3h5johcpk2ched6vitu8s539e6767122.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=s0tjiwReJk32LkCAbQYA2KoF9YZzkT&access_type=offline


gio: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=285282244977-3h5johcpk2ched6vitu8s539e6767122.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=s0tjiwReJk32LkCAbQYA2KoF9YZzkT&access_type=offline: Operation not supported


🌎 Drive a tiny robot in your stomach
From: Freethink Weekly <freethinkweekly@e.freethink.com>

Our Holiday Gift to you - Black Friday/Small Business Saturday Deal
From: Sushioki <noreply@tapmango.com>

Check this out on Amazon
From: Taylor <hearntaylor@gmail.com>

Your Amazon.com order #113-4865081-5196219 has shipped
From: "Amazon.com" <shipment-tracking@amazon.com>

Delivered: Your Amazon.com order #113-7586029-0817801
From: "Amazon.com" <order-update@amazon.com>

A provider is ready for you
From: Amazon Health <hello@email.health.amazon.com>

USPS® Item Delivered, Front Desk/Reception/Mail Room 9400101699330010406808
From: auto-reply@usps.com

Superconductivity and neural network pioneer Leon Cooper dies at 94
From: "Nature Briefing: AI & Robotics" <ai-briefing@nature.com>

Your Amazon.com order #111-9723940-0220208 has shipped
From: "Amazon.com" <shipment-tracking@amazon.com>

Level up your docs with visuals
From: Canva <marketing@engage.canva.com>

Gifts for him under $50
From: Po

In [4]:
email_summarizer = EmailSummarizer(Model.HAIKU)
for i in range(max_emails):
    pprint.pprint(email_summarizer.summarize(unread_inbox[i]), width=140)

EmailSummary(email_id='1936b0941ad9cc83',
             priority=<EmailPriority.MEDIUM: 'MEDIUM'>,
             email_type='Newsletters',
             summary='This email is a weekly newsletter from Freethink, a media company that covers emerging technologies and scientific '
                     'advancements. The main content of the email includes:\n'
                     '\n'
                     '- Information about a tiny robot called Pillbot that can be swallowed and controlled remotely to explore the '
                     'digestive tract. This technology could potentially be used for medical procedures in the future.\n'
                     '- An article about a startup company called Boom that is working to revive supersonic air travel, with the goal of '
                     'creating a plane that can fly twice as fast as current commercial airliners.\n'
                     '- An article about the legal battle between the Internet Archive and publishers over the "fair use" o

In [5]:
# email_briefing_generator = EmailBriefingGenerator(Model.SONNET, email_summarizer)
email_briefing_generator = EmailBriefingGenerator(Model.HAIKU, email_summarizer)
briefing = email_briefing_generator.generate_briefing(unread_inbox)

In [6]:
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

console = Console()
console.print(Panel(
    Markdown(briefing),
    title="[bold cyan]Email Briefing[/bold cyan]",
    border_style="bright_blue"
))

╭──────────────────────────────────────────────── Email Briefing ─────────────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                              Executive Summary                                              ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│ This daily briefing provides an overview of key information and action items from the user's email              │
│ communications. The main topics covered include emerging technologies, retail promotions, shipping updates,     │
│ healthcare services, financial updates, and account security.                                                   │
│                                                                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                            Emerging Technologies                                            ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│ [<email_id>1936b0941ad9cc83</email_id>] The user received a newsletter from Freethink, a media company covering │
│ science and technology. The newsletter included information about a tiny robot called Pillbot that can be       │
│ swallowed and controlled remotely to explore the digestive tract, a startup called Boom that is working to      │
│ revive supersonic air travel, and a legal battle between the Internet Archive and publishers over the "fair     │
│ use" of copyrighted books. While these topics are interesting, there are no immediate deadlines or required     │
│ actions.                                                                                                        │
│                                                                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                              Retail Promotions                                              ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│ [<email_id>1936aee58578c4d8</email_id>], [<email_id>1936ab92cae06ccd</email_id>],                               │
│ [<email_id>19368adbeefa7ee4</email_id>] The user received several marketing emails promoting various retail     │
│ deals and offerings:                                                                                            │
│                                                                                                                 │
│  • Sushioki is offering a BOGO 1/2 off special offer for Black Friday/Small Business Saturday.                  │
│  • WITHINGS is offering a limited-time deal on their Smart Scales with full body analysis.                      │
│  • Poshmark is showcasing gift ideas under $50, including Harley Davidson T-shirts, denim jackets, and Yeti     │
│    tumblers.                                                                                                    │
│                                                                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                              Shippin

In [7]:
inbox_manager = InboxManager(Model.HAIKU, email_summarizer)
inbox_manager.manage_inbox(unread_inbox)

KeyError: 'reasoning'